# 12장 FP-성장 알고리즘으로 빈발 아이템 집합 찾기
---
    검색어에서 빈발하게 함께 발생하는 단어
    어프라이어리 알고리즘: 빈발 아이템 집합, 연관규칙
    FP-성장 알고리즘: 빈발 아이템 집합 => 빠른 수행속도
        1. FP-트리 구축
        2. FP-트리를 이용해 빈발 아이템 집합 마이닝


## 12.1 FP-트리: 데이터 집합을 부호화하는 효과적인 방법
---
    * FP-성장 (Frequent Pattern)
    장점: 어프라이어리보다 빠르다.
    단점: 구현하기 어렵다.
![img](1.png)  
![img](2.png)  

    - 발생이 잦은 아이템 집합을 저장    
    cf) 트리 + 링크드 리스트
    - 트리와의 차이점: 같은 트리에 여러번 표현될 수 있음
    - 노드 링크(node link): 유사한 아이템간 연결
    최소 지지도(출현하는 횟수) 설정


## 12.2 FP-트리 구축하기
---

### 12.2.1 FP트리 데이터 구조 생성하기
---


In [1]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = {} 
    
    def inc(self, numOccur):
        self.count += numOccur
        
    def disp(self, ind=1):
        print '  '*ind, self.name, ' ', self.count
        for child in self.children.values():
            child.disp(ind+1)


In [2]:
rootNode = treeNode('pyramid', 9, None)
rootNode.children['eye'] = treeNode('eye', 13, None)
rootNode.children['pheonix'] = treeNode('phoenix', 3, None)
rootNode.disp()

   pyramid   9
     eye   13
     phoenix   3


### 12.2.2 FP-트리 구축하기
---
![img](3.png)
    * 헤더테이블(Header Table)
    FP트리의 원소들에게 빠르게 접근할 수 있도록 만듬
    딕셔너리 구조 - pointer저장
    
    * FP트리 만들기
    1. 각 아이템의 빈도를 셈한 후 최소지지도를 만족하지 못하는 아이템들을 제거 / 빈도순으로 정렬
    2. 각 아이템 집합이 존재하지 않는다면 새로우 경로를 생성
    3. 각 아이템 집합이 이미 존재한다면 이 아이템 집합을 기존 경로에 추가
    
    1.각 아이템의 빈도를 셈한 후 최소지지도를 만족하지 못하는 아이템들을 제거 / 빈도순으로 정렬
![img](4.png)
    
    2. 각 아이템 집합이 존재하지 않는다면 새로우 경로를 생성 / 3. 각 아이템 집합이 이미 존재한다면 이 아이템 집합을 기존 경로에 추가
![img](5.png)


In [3]:
def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    headerTable = {}
    #go over dataSet twice
    for trans in dataSet:#first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in headerTable.keys():  #remove items not meeting minSup
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    #print 'freqItemSet: ',freqItemSet
    if len(freqItemSet) == 0: return None, None  #if no items meet min support -->get out
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None) #create tree
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = {}
        for item in tranSet:  #put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)#populate tree with ordered freq itemset
    return retTree, headerTable #return tree and header table

def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #update header table 
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:#call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode


In [4]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [58]:
simpDat = loadSimpDat()
print simpDat

[['r', 'z', 'h', 'j', 'p'], ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'], ['z'], ['r', 'x', 'n', 'o', 's'], ['y', 'r', 'x', 'z', 'q', 't', 'p'], ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]


In [59]:
initSet = createInitSet(simpDat)
print initSet

{frozenset(['e', 'm', 'q', 's', 't', 'y', 'x', 'z']): 1, frozenset(['x', 's', 'r', 'o', 'n']): 1, frozenset(['s', 'u', 't', 'w', 'v', 'y', 'x', 'z']): 1, frozenset(['q', 'p', 'r', 't', 'y', 'x', 'z']): 1, frozenset(['h', 'r', 'z', 'p', 'j']): 1, frozenset(['z']): 1}


In [60]:
myFPtree,myHeaderTab = createTree(initSet, 3)
print myFPtree.disp()

   Null Set   1
     x   1
       s   1
         r   1
     z   5
       x   3
         y   3
           s   2
             t   2
           r   1
             t   1
       r   1
None


In [61]:
print myHeaderTab
print
print myHeaderTab['r']
print
print myHeaderTab['r'][1]
print
print myHeaderTab['r'][1].nodeLink
print
print myHeaderTab['r'][1].nodeLink.nodeLink
print
print myHeaderTab['r'][1].nodeLink.nodeLink.nodeLink

{'s': [3, <__main__.treeNode instance at 0x1082e8b00>], 'r': [3, <__main__.treeNode instance at 0x1082e8b90>], 't': [3, <__main__.treeNode instance at 0x1082e8b48>], 'y': [3, <__main__.treeNode instance at 0x1082e8560>], 'x': [4, <__main__.treeNode instance at 0x1082e8f80>], 'z': [5, <__main__.treeNode instance at 0x1082e8f38>]}

[3, <__main__.treeNode instance at 0x1082e8b90>]

<__main__.treeNode instance at 0x1082e8b90>

<__main__.treeNode instance at 0x1082e8cb0>

<__main__.treeNode instance at 0x1082e8e60>

None


## 12.3 FP-트리로 빈발 아이템 집합 마이닝하기
---
    원래 데이터 집합을 필요로 하지 않는다!
    1. FP-트리로부터 기본 패턴 조건들을 구한다.
    2. 기본 패턴 조건을 가지고 FP-트리 조건을 구축한다.
    3. 트리가 하나의 아이템만을 포함할 때까지 1단계와 2단계를 재귀적으로 반복한다.
    
### 12.3.1 기본 패턴 조건 추출하기
---
    * 기본 패턴 조건(conditional pattern base)
    찾고자 하는 아이템이 포함된 말단부분에 대한 경로들의 집합
    
    각각의 경로: 사전 경로(prefix path)
![img](6.png)  

    사전 경로를 찾기 위해 전부 탐색해도 되지만 우리는 만들어놓은 헤더테이블이 있다!
    헤더테이블로 부터 시작해서 루트로 가는 경로는 모두 검색하자    

In [62]:
def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)
    
def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [10]:
findPrefixPath('x', myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [11]:
findPrefixPath('z', myHeaderTab['z'][1])

{}

In [12]:
findPrefixPath('r', myHeaderTab['r'][1])

{frozenset({'s', 'x'}): 1, frozenset({'z'}): 1, frozenset({'x', 'y', 'z'}): 1}

### 12.3.2 FP-트리 조건 생성하기
---
    방금 찾은 기본 패턴 조건(Conditional pattern base)을 바탕으로 다시 FP-트리 조건(Conditional FP-tree)를 형성한다(FP트리 조건)
    ex) t -> x, y, z

![img](7.png)  



In [66]:
findPrefixPath('t', myHeaderTab['t'][1])

{frozenset({'s', 'x', 'y', 'z'}): 2, frozenset({'r', 'x', 'y', 'z'}): 1}

In [68]:
bigL = [v[0] for v in sorted(myHeaderTab.items(), key=lambda p: p[1])]
print bigL

['y', 's', 't', 'r', 'x', 'z']


In [79]:
freqItems = []
for basePat in bigL:  #start from bottom of header table
    newFreqSet = set([]).copy()
    newFreqSet.add(basePat)
    print 'finalFrequent Item: ',newFreqSet
    freqItems.append(newFreqSet)
    condPattBases = findPrefixPath(basePat, myHeaderTab[basePat][1])
    print 'condPattBases :',basePat, condPattBases
    myCondTree, myHead = createTree(condPattBases, 3)
    print 'head from conditional tree: ', myHead
    if myHead != None: #3. mine cond. FP-tree
        print 'conditional tree for: ',newFreqSet
        myCondTree.disp(1)            
        mineTree(myCondTree, myHead, 3, newFreqSet, freqItems)

finalFrequent Item:  set(['y'])
condPattBases : y {frozenset(['x', 'z']): 3}
head from conditional tree:  {'x': [3, <__main__.treeNode instance at 0x108322b00>], 'z': [3, <__main__.treeNode instance at 0x108322b48>]}
conditional tree for:  set(['y'])
   Null Set   1
     x   3
       z   3
finalFrequent Item:  set(['y', 'x'])
condPattBases : x {}
head from conditional tree:  None
finalFrequent Item:  set(['y', 'z'])
condPattBases : z {frozenset(['x']): 3}
head from conditional tree:  {'x': [3, <__main__.treeNode instance at 0x108322bd8>]}
conditional tree for:  set(['y', 'z'])
   Null Set   1
     x   3
finalFrequent Item:  set(['y', 'x', 'z'])
condPattBases : x {}
head from conditional tree:  None
finalFrequent Item:  set(['s'])
condPattBases : s {frozenset(['y', 'x', 'z']): 2, frozenset(['x']): 1}
head from conditional tree:  {'x': [3, <__main__.treeNode instance at 0x108322c68>]}
conditional tree for:  set(['s'])
   Null Set   1
     x   3
finalFrequent Item:  set(['x', 's'])
condPa

In [82]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1])]#(sort header table)
    for basePat in bigL:  #start from bottom of header table
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
#         print 'finalFrequent Item: ',newFreqSet    #append to set
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
#         print 'condPattBases :',basePat, condPattBases
        #2. construct cond FP-tree from cond. pattern base
        myCondTree, myHead = createTree(condPattBases, minSup)
#         print 'head from conditional tree: ', myHead
        if myHead != None: #3. mine cond. FP-tree
            print 'conditional tree for: ',newFreqSet
            myCondTree.disp(1)            
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [84]:
freqItems = []
mineTree(myFPtree, myHeaderTab, 3, set(['t']), freqItems)

conditional tree for:  set(['y', 't'])
   Null Set   1
     x   3
       z   3
conditional tree for:  set(['y', 'z', 't'])
   Null Set   1
     x   3
conditional tree for:  set(['s', 't'])
   Null Set   1
     x   3
conditional tree for:  set(['t'])
   Null Set   1
     y   3
       x   3
         z   3
conditional tree for:  set(['z', 't'])
   Null Set   1
     y   3
       x   3
conditional tree for:  set(['x', 'z', 't'])
   Null Set   1
     y   3
conditional tree for:  set(['x', 't'])
   Null Set   1
     y   3
conditional tree for:  set(['x', 't'])
   Null Set   1
     z   3


In [86]:
freqItems

[{'t', 'y'},
 {'t', 'y', 'z'},
 {'t', 'x', 'y', 'z'},
 {'t', 'x', 'y'},
 {'s', 't'},
 {'s', 't', 'x'},
 {'t'},
 {'t', 'z'},
 {'t', 'x', 'z'},
 {'t', 'x', 'y', 'z'},
 {'t', 'y', 'z'},
 {'t', 'x'},
 {'t', 'x', 'y'},
 {'t', 'y'},
 {'r', 't'},
 {'t', 'x'},
 {'t', 'x', 'z'},
 {'t', 'z'}]

## 12.4 예제: 트위터 피드에서 함께 발생하는 단어 찾기
---

In [15]:
import twitter
from time import sleep
import re

def textParse(bigString):
    urlsRemoved = re.sub('(http:[/][/]|www.)([a-z]|[A-Z]|[0-9]|[/.]|[~])*', '', bigString)    
    listOfTokens = re.split(r'\W*', urlsRemoved)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

def getLotsOfTweets(searchStr):
    CONSUMER_KEY = 't3TWVLZA8kqbGYEA3MjoOHYmK'
    CONSUMER_SECRET = 'Vd6p38wRo0Bzw1Mc0pPfSNeN5QrzjK36Lzg42vlGmXFYt747vY'
    ACCESS_TOKEN_KEY = '108031820-vNx453wg1g8w35QtCoiQvOYQxRPOqF38umAmOpRG'
    ACCESS_TOKEN_SECRET = 'NA4Kw2piABZjWBTX1O7CSbq5SOAn1WhuPqrBYxNvmX59a'
    api = twitter.Api(consumer_key=CONSUMER_KEY, consumer_secret=CONSUMER_SECRET,
                      access_token_key=ACCESS_TOKEN_KEY, 
                      access_token_secret=ACCESS_TOKEN_SECRET)
    #you can get 1500 results 15 pages * 100 per page
    resultsPages = []
    for i in range(1,15):
        print "fetching page %d" % i
        searchResults = api.GetSearch(searchStr)
        resultsPages.append(searchResults)
        sleep(6)
    return resultsPages

def mineTweets(tweetArr, minSup=5):
    parsedList = []
    for i in range(14):
        for j in range(15):
            parsedList.append(textParse(tweetArr[i][j].text))
    initSet = createInitSet(parsedList)
    myFPtree, myHeaderTab = createTree(initSet, minSup)
    myFreqList = []
    mineTree(myFPtree, myHeaderTab, minSup, set([]), myFreqList)
    return myFreqList

#minSup = 3
#simpDat = loadSimpDat()
#initSet = createInitSet(simpDat)
#myFPtree, myHeaderTab = createTree(initSet, minSup)
#myFPtree.disp()
#myFreqList = []
#mineTree(myFPtree, myHeaderTab, minSup, set([]), myFreqList)


In [16]:
#-*- coding: utf-8 -*-
lotsOtweet = getLotsOfTweets('박근혜')

fetching page 1
fetching page 2
fetching page 3
fetching page 4
fetching page 5
fetching page 6
fetching page 7
fetching page 8
fetching page 9
fetching page 10
fetching page 11
fetching page 12
fetching page 13
fetching page 14


In [37]:
#-*- coding: utf-8 -*-
for i in lotsOtweet:
    for j in i:
        print j.text

에너지 공공성과 정의를 훼손하는 박근혜 정부의 일방적인 에너지산업 민영화에 반대합니다!  https://t.co/RoUFvE29rY
&lt;반헌법행위자열전&gt; 편찬위원회(편찬위)가  열전에 수록될 가능성이 높은 ‘집중검토 대상자 405명’의 명단을 발표했습니다. 박근혜, 황교안, 양승태.... 
https://t.co/9HRTq5I40U
"박근혜 즉각 탄핵" 

2월 18일(토) 광화문 광장
16차 범국민행동의 날.

정의당 당원분들은 15시 30분까지 세종문화회관 계단 앞으로 모여주시길 부탁드립니다. https://t.co/2I4gLbU7as
RT @foreverphoenix7: 박사모집회에 
독립군이 나타났다

당신들이 잘못해서
야단치러 왔다며
호통치는 어르신

태극기 대신 박근혜
얼굴을 흔들어라

독립운동가들이 흔들던
태극기를 왜 범죄자를
위해 흔드냐!

핵사이다 어르신 만세!!! h…
RT @sarangterpb: @mediamongu 

박근혜탄핵 반대자
새누리당 최경환 지역구네요.
흠..

문명고등학교
김태동 교장은 속히 
국정교과서 신청을 철회하길..
RT @foreverphoenix7: 박사모집회에 
독립군이 나타났다

당신들이 잘못해서
야단치러 왔다며
호통치는 어르신

태극기 대신 박근혜
얼굴을 흔들어라

독립운동가들이 흔들던
태극기를 왜 범죄자를
위해 흔드냐!

핵사이다 어르신 만세!!! h…
RT @lettresdeparis: #횃불 159차

부정선거 박근혜를 청와대에서 구출하라!
내란사범 박근혜를 감옥으로 당장 이송하라!

#개표조작 내란사범 박근혜를 체포/구속하라! #부정선거 살인정권 박근혜를 감옥으로! https://t.co/dX…
[“MBC, 박근혜 낙점한 청부 사장 안돼”]
MBC 뉴스데스크의 시청률은 5%대로 곤두박질 쳤다.
https://t.co/RzJ2glHiJa https://t.co/P7FlT4EqNx
RT @usminjooforum: 한정석 판사는 위대한 인물이다…”엄지 척”

박근혜와 이재용의 구속은 정의의 출발점

In [34]:
#-*- coding: utf-8 -*-
lotsOtweet2 = getLotsOfTweets('Trump')

fetching page 1
fetching page 2
fetching page 3
fetching page 4
fetching page 5
fetching page 6
fetching page 7
fetching page 8
fetching page 9
fetching page 10
fetching page 11
fetching page 12
fetching page 13
fetching page 14


In [39]:
#-*- coding: utf-8 -*-
for i in lotsOtweet2:
    for j in i:
        print j.text

TIME’s new cover: Inside Donald Trump's White House chaos https://t.co/hctIFEcOSG https://t.co/1hSQNrY6JQ
Wow watch Shepard Smith’s reaction to Trump’s “fake news” claims at press conference https://t.co/s0DC9GRul9
oh my god somebody modeled the trump handshake https://t.co/2L1Sst5lFm
RT @Sky1876Online: Christie will work in private sector, not W.H. https://t.co/YgAWhuZd01 #News #Trump #Christie #private #sector

https://…
RT @LindaSuhler: Promises KEPT...
Trump Welcomes Wife of Venezuelan Political Prisoner, Keeping Vow to Oppressed Latinos
#MAGA 
https://t.c…
RT @LawyerRogelio: I've enjoyed this video so much i'm uploading again.  UNTIL THEIR ATTITUDE CHANGES, WE  WILL NOT WORK WITH TRUMP SUPPORT…
Sign the petition: Tell Senate Democrats to protect our consumer protection agency from Trump https://t.co/FULfip21nS via @CREDOMobile #p2
Global alarm, Israeli joy at Trump two-state shift https://t.co/hnQq7dj0pZ
RT @pulzo: Ratifican condena contra Leopoldo López, pese a apoyo de Trump htt

In [55]:
listOfTerms = mineTweets(lotsOtweet2, 5)
len(listOfTerms)
print listOfTerms

finalFrequent Item:  set([u'after'])
condPattBases : after {frozenset([u'conference', u'trump', u'this', u'https', u'press', u'president', u'the']): 1, frozenset([u'this', u'his', u'trump', u'https']): 1, frozenset([u'president', u'donald', u'the', u'trump']): 1, frozenset([u'this', u'president', u'trump']): 1, frozenset([u'news', u'trump', u'https']): 1}
head from conditional tree:  {u'trump': [5, <__main__.treeNode instance at 0x104bdfea8>]}
conditional tree for:  set([u'after'])
   Null Set   1
     trump   5
finalFrequent Item:  set([u'after', u'trump'])
condPattBases : trump {}
head from conditional tree:  None
finalFrequent Item:  set([u'just'])
condPattBases : just {frozenset([u'conference', u'and', u'trump', u'https', u'press', u'with']): 1, frozenset([u'trump', u'they']): 1, frozenset([u'the', u'trump']): 1, frozenset([u'the', u'trump', u'they']): 1, frozenset([u'and', u'donald', u'trump']): 1}
head from conditional tree:  {u'trump': [5, <__main__.treeNode instance at 0x1082f4

맵 리듀스 방법과 유사!

## 12.5 예제: 뉴스 사이트에서 클릭 스트림 마이닝하기
---

In [41]:
parsedDat = [line.split() for line in open('kosarak.dat').readlines()]

In [42]:
initSet = createInitSet(parsedDat)

In [45]:
from time import time
t0 = time()
myFPtree, myHeaderTab = createTree(initSet, 100000)
print "Tree creating time:", round(time()-t0, 3), "s"

Tree creating time: 14.528 s


In [56]:
for i in myFPtree.children:
    print myFPtree.children[i].name

11
1
3
6


In [57]:
mineTree(myFPtree, myHeaderTab, 3, set([]), freqItems)

finalFrequent Item:  set(['1'])
condPattBases : 1 {frozenset(['11', '3']): 1565, frozenset(['11', '3', '6']): 34141, frozenset(['11']): 1882, frozenset(['3']): 12917, frozenset(['6']): 16461, frozenset(['11', '6']): 43366, frozenset(['3', '6']): 13436}
head from conditional tree:  {'11': [80954, <__main__.treeNode instance at 0x148a61ea8>], '3': [62059, <__main__.treeNode instance at 0x148a618c0>], '6': [107404, <__main__.treeNode instance at 0x148a61b90>]}
conditional tree for:  set(['1'])
   Null Set   1
     11   3447
       3   1565
     3   12917
     6   107404
       11   77507
         3   34141
       3   13436
finalFrequent Item:  set(['1', '3'])
condPattBases : 3 {frozenset(['11', '6']): 34141, frozenset(['11']): 1565, frozenset(['6']): 13436}
head from conditional tree:  {'11': [35706, <__main__.treeNode instance at 0x1075948c0>], '6': [47577, <__main__.treeNode instance at 0x108312170>]}
conditional tree for:  set(['1', '3'])
   Null Set   1
     11   1565
     6   47577
 